# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 21 2022 11:50AM,240083,18,CLRSPC-21101953,"ClearSpec, LLC",Released
1,Jun 21 2022 11:44AM,240081,20,7912021-BO,"ACI Healthcare USA, Inc.",Released
2,Jun 21 2022 11:44AM,240081,20,7918049-BO,"ACI Healthcare USA, Inc.",Released
3,Jun 21 2022 11:44AM,240081,20,7952198-BO,"ACI Healthcare USA, Inc.",Released
4,Jun 21 2022 11:44AM,240081,20,ACI-8057010-BO,"ACI Healthcare USA, Inc.",Released
5,Jun 21 2022 11:44AM,240081,20,8091764-BO,"ACI Healthcare USA, Inc.",Released
6,Jun 21 2022 11:44AM,240081,20,8071811-BO1,"ACI Healthcare USA, Inc.",Released
7,Jun 21 2022 11:44AM,240081,20,ACI-8109990-BO,"ACI Healthcare USA, Inc.",Released
8,Jun 21 2022 11:44AM,240081,20,ACI-8091181-BO1,"ACI Healthcare USA, Inc.",Released
9,Jun 21 2022 11:44AM,240081,20,8118233-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
54,240078,Released,1
55,240079,Released,12
56,240080,Released,1
57,240081,Released,37
58,240083,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240078,NaN,NaN,1.0
240079,NaN,NaN,12.0
240080,NaN,NaN,1.0
240081,NaN,NaN,37.0
240083,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239959,0.0,0.0,1.0
239965,0.0,1.0,0.0
239982,29.0,9.0,2.0
239992,0.0,0.0,1.0
240002,0.0,8.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239959,0,0,1
239965,0,1,0
239982,29,9,2
239992,0,0,1
240002,0,8,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239959,0,0,1
1,239965,0,1,0
2,239982,29,9,2
3,239992,0,0,1
4,240002,0,8,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239959,,,1
1,239965,,1,
2,239982,29,9,2
3,239992,,,1
4,240002,,8,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 21 2022 11:50AM,240083,18,"ClearSpec, LLC"
1,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc."
38,Jun 21 2022 11:41AM,240080,10,"Methapharm, Inc."
39,Jun 21 2022 11:29AM,240079,10,ISDIN Corporation
51,Jun 21 2022 11:29AM,240077,20,Alumier Labs Inc.
90,Jun 21 2022 11:25AM,240078,10,"Citieffe, Inc."
91,Jun 21 2022 11:18AM,240076,10,MedStone Pharma LLC
103,Jun 21 2022 11:13AM,240075,10,Emerginnova
104,Jun 21 2022 11:10AM,240072,10,"Methapharm, Inc."
113,Jun 21 2022 10:51AM,240070,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 21 2022 11:50AM,240083,18,"ClearSpec, LLC",,,1
1,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc.",,,37
2,Jun 21 2022 11:41AM,240080,10,"Methapharm, Inc.",,,1
3,Jun 21 2022 11:29AM,240079,10,ISDIN Corporation,,,12
4,Jun 21 2022 11:29AM,240077,20,Alumier Labs Inc.,,,39
5,Jun 21 2022 11:25AM,240078,10,"Citieffe, Inc.",,,1
6,Jun 21 2022 11:18AM,240076,10,MedStone Pharma LLC,,,12
7,Jun 21 2022 11:13AM,240075,10,Emerginnova,,,1
8,Jun 21 2022 11:10AM,240072,10,"Methapharm, Inc.",,,9
9,Jun 21 2022 10:51AM,240070,16,Sartorius Bioprocess Solutions,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 11:50AM,240083,18,"ClearSpec, LLC",1,,
1,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc.",37,,
2,Jun 21 2022 11:41AM,240080,10,"Methapharm, Inc.",1,,
3,Jun 21 2022 11:29AM,240079,10,ISDIN Corporation,12,,
4,Jun 21 2022 11:29AM,240077,20,Alumier Labs Inc.,39,,
5,Jun 21 2022 11:25AM,240078,10,"Citieffe, Inc.",1,,
6,Jun 21 2022 11:18AM,240076,10,MedStone Pharma LLC,12,,
7,Jun 21 2022 11:13AM,240075,10,Emerginnova,1,,
8,Jun 21 2022 11:10AM,240072,10,"Methapharm, Inc.",9,,
9,Jun 21 2022 10:51AM,240070,16,Sartorius Bioprocess Solutions,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 11:50AM,240083,18,"ClearSpec, LLC",1,,
1,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc.",37,,
2,Jun 21 2022 11:41AM,240080,10,"Methapharm, Inc.",1,,
3,Jun 21 2022 11:29AM,240079,10,ISDIN Corporation,12,,
4,Jun 21 2022 11:29AM,240077,20,Alumier Labs Inc.,39,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 11:50AM,240083,18,"ClearSpec, LLC",1.0,NaN,NaN
1,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc.",37.0,NaN,NaN
2,Jun 21 2022 11:41AM,240080,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Jun 21 2022 11:29AM,240079,10,ISDIN Corporation,12.0,NaN,NaN
4,Jun 21 2022 11:29AM,240077,20,Alumier Labs Inc.,39.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 11:50AM,240083,18,"ClearSpec, LLC",1.0,0.0,0.0
1,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc.",37.0,0.0,0.0
2,Jun 21 2022 11:41AM,240080,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Jun 21 2022 11:29AM,240079,10,ISDIN Corporation,12.0,0.0,0.0
4,Jun 21 2022 11:29AM,240077,20,Alumier Labs Inc.,39.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5760948,286.0,24.0,34.0
12,240055,0.0,2.0,0.0
15,1680408,58.0,45.0,0.0
16,240070,2.0,0.0,0.0
18,480108,2.0,0.0,0.0
19,1200051,10.0,10.0,2.0
20,720165,77.0,0.0,0.0
21,239992,1.0,0.0,0.0
22,240003,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5760948,286.0,24.0,34.0
1,12,240055,0.0,2.0,0.0
2,15,1680408,58.0,45.0,0.0
3,16,240070,2.0,0.0,0.0
4,18,480108,2.0,0.0,0.0
5,19,1200051,10.0,10.0,2.0
6,20,720165,77.0,0.0,0.0
7,21,239992,1.0,0.0,0.0
8,22,240003,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,286.0,24.0,34.0
1,12,0.0,2.0,0.0
2,15,58.0,45.0,0.0
3,16,2.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,10.0,10.0,2.0
6,20,77.0,0.0,0.0
7,21,1.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,286.0
1,12,Released,0.0
2,15,Released,58.0
3,16,Released,2.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,34.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
Executing,24.0,2.0,45.0,0.0,0.0,10.0,0.0,0.0,0.0
Released,286.0,0.0,58.0,2.0,2.0,10.0,77.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,34.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,24.0,2.0,45.0,0.0,0.0,10.0,0.0,0.0,0.0
2,Released,286.0,0.0,58.0,2.0,2.0,10.0,77.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,34.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,24.0,2.0,45.0,0.0,0.0,10.0,0.0,0.0,0.0
2,Released,286.0,0.0,58.0,2.0,2.0,10.0,77.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()